# **Focus** : How to simplify the shape of a line/polygon ? 
![Alt text](https://raw.githubusercontent.com/gowestmen/images/refs/heads/main/line_simp.png)
## Detailed explaination of some generalization algorithms
Press **`space bar`** to display the next slide or use click arrows on the bottom right.
_If this Notebook doesn't appear in slideshow mode, reload the page_

- 🙋 **For whom ?** : 
  - people who have spatial data represented by lines or polygones and wish to simplify its aspect.
  - expert cartographers wishing to discover new simplification algorithms offered by the CartAGen tool
  - student in algorithm class 

Press **`spacebar`** to display the following message

You may need to **`scroll down`** to see it.
 _Reload the page if you can't scroll down._ 


- ❓ **What is CartAGen ?** :
    - an **open source Python library** made by **IGN** (i.e the french national mapping agency) researchers
    - a tool to carry out cartographic generalization processes and **automate** them
    - a **QGIS plugin** to use the Python library in a GIS environment  
    
Press **`space bar`** and **`scroll down`** to show next message 


- 📗 **How to use this Notebook ?** : 
    - a Notebook is a document that mixes **markdown cells** (i.e. cells with formatted text like this one) and **code cells**, which you can modify and execute at will

Press **`right arrow`** to show next slide and start part 0, or press **`space bar`** to show sub-slide and see useful links

   
- 🔗 **Useful links** : 
    - [Library repository](https://github.com/LostInZoom/cartagen4py?tab=readme-ov-file) 
    - [QGIS Plugin repository](https://github.com/LostInZoom/cartagen-qgis)
    - [CartAGen documentation](https://cartagen.readthedocs.io/en/latest/)

_by Paul Bourcier, 2024 - EUPL 1.2_

Press **`space bar`** to show next slide, press **`shift + space bar`** to go backward\
**`Click`** question mark icon to show other shortcuts

## ⚙️ **Part 0 :** Preparing the working environment

Press **`space bar`** to show code cell, then press **`shift + enter`** to execute it\
**`*`** symbol means code output is loading

In [ ]:
# Importing libraries
import cartagen as cg # the main library, containing cartographic generalization algorithms

#geographic data manipulation 
import geopandas as gp # used to import and manipulate geographic data
from shapely.geometry import Polygon, Point, LineString, MultiPoint # Create and manipulate geometries
from shapely.ops import transform
from shapely import line_interpolate_point, centroid, force_2d, intersection

#displaying the data
from matplotlib import pyplot as plt # for ploting results, especially maps

#creating interactive outputs
import ipywidgets # creation of interactive plots
from IPython.display import display

# other data manipulation
import numpy as np # Using mathematical operations in Python
import random #generating random numbers

import warnings # Remove warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*GeoDataFrame you are attempting to plot is empty.*") # Remove user warning 
warnings.filterwarnings("ignore", category=UserWarning, message=".*CRS not set for some of the concatenation inputs.*.")

**Cartogaphic generalisation** refers to the process of transforming the graphic representation of map elements in order to reduce their complexity.

Here are some examples of generalisation techniques : 


![Alt text](https://raw.githubusercontent.com/gowestmen/images/refs/heads/main/carto_generalization_operators.png)

The need for cartographic generalization can arise in a number of situations: 
- When the map is illegible because it contains too many elements or the elements are too small.
- In the context of sensitive spatial data (e.g. hospital patient addresses), to avoid being able to identify specific locations.
- For aesthetic purposes
- To reduce the complexity/weight of spatial data prior to further geoprocessing

When it comes to vector data in the form of a line or polygon, which can represent features such as roads, rivers, administrative boundaries, etc., cartographic generalization often involves simplifying the geometry.

In **CartAGen**, there are currently four algorithms available dedicated to these techniques. Each of these algorithms has its own specificities, making them more or less suitable depending on the feature represented. Some specialize in generalizing rivers, while others seem more suited to roads.

To choose wisely which of these algorithms is best suited to your data, it's important to understand how they work! That's what this Notebook aims to show you.
Here are the algorithms reviewed : 
- Douglas-Peucker algorithm
- Visvalingam-Whyatt algorithm
- Raposo algorithm
- Li-Openshaw algorithm 

##  **Part 1** : Douglas-Peucker algorithm

This line simplification algorithm was created by [David H. Douglas and Thomas K. Peucker](https://utpjournals.press/doi/10.3138/FM57-6770-U75U-7727) in 1973. It is the oldest of the four presented here.

It is a line filtering algorithm, which means that it filters the vertices of the line (or polygon) to only retain the most important ones to preserve the shape of the line. The algorithm iteratively searches the most characteristics vertices of portions of the line and decides to retain or remove them given a distance threshold.

**1.1/** Load the line to simplify

In [ ]:
line_1 = gp.read_file("data/data_lines/ligne.geojson")
line_1_vertices = gp.GeoDataFrame(geometry=gp.GeoSeries([Point(vertice) for vertice in line_1.geometry[0].coords]))
load_vertices = ipywidgets.Checkbox(
    value=False,
    description='Show vertices'
)
figure_size = ipywidgets.IntSlider(value =6, min = 1, max = 16,step = 1,continuous_update=False)

def load_line_1(figure_size, load_vertices):
    ax = line_1.plot(figsize=(figure_size,figure_size))
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    if load_vertices == True:  
        line_1_vertices.plot(ax=ax)

out_load_line_1 = ipywidgets.interactive(load_line_1, figure_size = figure_size, load_vertices = load_vertices)
display(out_load_line_1)

**1.2/** Principle

In [ ]:
next_step = ipywidgets.Button(description='Next step',button_style='')
back_step = ipywidgets.Button(description='Previous step',button_style='')
count = ipywidgets.IntText(value=0, min=0, max=8,disabled=True)

def dp(step):
    global pt_red
    global pt_green
    global second_line_1
    global second_line_2
    global farthest_pt_1
    global farthest_pt_2
    global third_line_1
    global third_line_2
    global fourth_line_1
    global fourth_line_2
    global gdf_red
    global gdf_green
    if count.value == 0:
        ax = line_1.plot(color="grey",figsize=(12,12))
        line_1_vertices.plot(ax=ax,color="grey")
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    if count.value == 1:
        ax = line_1.plot(color="grey",figsize=(12,12))
        line_1_vertices.plot(ax=ax,color="grey")
        gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([line_1_vertices.geometry[0],line_1_vertices.geometry[len(line_1_vertices)-1]]))).plot(ax=ax)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        print("Retrieve the line formed by the first and last point")
    if count.value == 2:
        print("Check for vertices above the length threshold")
        ax = line_1.plot(color="grey",figsize=(12,12))
        buff = gp.GeoSeries(LineString([line_1_vertices.geometry[0],line_1_vertices.geometry[len(line_1_vertices)-1]])).buffer(13,cap_style='flat')
        buff.plot(ax=ax,color = "orange", alpha = 0.5)
        gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([line_1_vertices.geometry[0],line_1_vertices.geometry[len(line_1_vertices)-1]]))).plot(ax=ax)
        pt_red = []
        pt_green = []
        for i in range(len(line_1_vertices)):
            pt = line_1_vertices[(line_1_vertices.index == i)].geometry
            pt = pt.reset_index(drop=True)
            
            if pt.intersects(buff.geometry)[0]:
                pt.plot(ax=ax,color = "grey")
                pt_red.append(pt)
            else: 
                pt.plot(ax=ax,color = "red")
                pt_green.append(pt)
        gdf_red = gp.GeoDataFrame(geometry = gp.GeoSeries([point[0] for point in pt_red]))
        gdf_green = gp.GeoDataFrame(geometry = gp.GeoSeries([point[0] for point in pt_green]))
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    if count.value == 3:
        print("As there are vertices above the length threshold, \nwe calculate two new lines with the point perpendicularly furthest from the previous line.")
        first_line = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([line_1_vertices.geometry[0],line_1_vertices.geometry[len(line_1_vertices)-1]])))
        ax = first_line.plot(linestyle = 'dashed',figsize=(12,12))
        line_1.plot(color="grey",ax=ax, alpha=0.5)
        gdf_red.plot(ax=ax, color='grey', alpha=0.5)
        gdf_green.plot(ax=ax, color='grey', alpha=0.5)
    
        gdf_green['distance'] = gdf_green.geometry.apply(lambda point: point.distance(first_line.geometry[0]))
        farthest_pt_1 = gdf_green[(gdf_green['distance'] == max(gdf_green['distance']))].geometry[gdf_green[(gdf_green['distance'] == max(gdf_green['distance']))].geometry.index[0]]
        second_line_1 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([line_1_vertices.geometry[0],farthest_pt_1])))
        second_line_2 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([farthest_pt_1,line_1_vertices.geometry[len(line_1_vertices)-1]])))
        second_line_1.plot(ax=ax)
        second_line_2.plot(ax=ax)
        
        seg_length = np.sqrt(abs(gdf_green[(gdf_green['distance'] == max(gdf_green['distance']))]['distance'].iloc[0]**2-second_line_1.geometry.length.iloc[0]**2))
        perpen_line = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([farthest_pt_1,line_interpolate_point(first_line, seg_length).geometry[0]])))
        perpen_line.plot(ax=ax,linestyle='dashed')
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    if count.value == 4:
        pt_green_2 = []
        pt_red_2 = []
        print("Checking the vectrice - line distance like in step 2, but with the new computed lines")
        ax = line_1.plot(color="grey",figsize=(12,12))
        buff_2 = second_line_1.geometry.buffer(13,cap_style='flat')
        buff_2.plot(ax=ax,color = "orange", alpha = 0.5)
        buff_3 = second_line_2.geometry.buffer(13,cap_style='flat')
        buff_3.plot(ax=ax,color = "orange", alpha = 0.5)
        second_line_1.plot(ax=ax)
        second_line_2.plot(ax=ax)
        counter = 0
        for i in range(len(line_1_vertices)):
            pt = line_1_vertices[(line_1_vertices.index == i)].geometry
            pt = pt.reset_index(drop=True)
            counter += 1
            if pt.intersects(buff_2)[0] or pt.intersects(buff_3)[0]:
                pt.plot(ax=ax,color = "grey")
                pt_red_2.append(pt)
            else: 
                pt.plot(ax=ax,color = "red")
                pt_green_2.append(pt)
        pt_green = [pt[0] for pt in pt_green_2]
        pt_red = [pt[0] for pt in pt_red_2]
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        gdf_red = gp.GeoDataFrame(geometry = pt_red)
        gdf_green = gp.GeoDataFrame(geometry = pt_green)
    if count.value == 5:
        print("Compute new lines since there are vertices above length threshold.")
        second_line_1 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([line_1_vertices.geometry[0],farthest_pt_1])))
        second_line_2 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([farthest_pt_1,line_1_vertices.geometry[len(line_1_vertices)-1]])))
        ax = second_line_1.plot(linestyle = 'dashed',figsize=(12,12))
        second_line_2.plot(ax=ax,linestyle = 'dashed')
        line_1.plot(color="grey",ax=ax, alpha = 0.5)
        line_1_vertices.plot(ax=ax, color='grey', alpha=0.5)
        
        gdf_green['distance'] = gdf_green.geometry.apply(lambda point: point.distance(second_line_1.geometry[0]))
        farthest_pt_2 = pt_green[0]
        third_line_1 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([line_1_vertices.geometry[0],farthest_pt_2])))
        third_line_1.plot(ax=ax)
        third_line_2 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([farthest_pt_2,farthest_pt_1])))
        third_line_2.plot(ax=ax)
        
        seg_length = np.sqrt(abs(gdf_green['distance'].iloc[0]**2-third_line_1.geometry.length.iloc[0]**2))
        perpen_line = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([farthest_pt_2,line_interpolate_point(second_line_1, seg_length).geometry[0]])))
        perpen_line.plot(ax=ax,linestyle='dashed')
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    if count.value == 6:
        pt_green_2 = []
        pt_red_2 = []
        print("Checking the vectrice - line distance like in step 2, but with the new computed lines")
        ax = line_1.plot(color="grey",figsize=(12,12))
        buff_4 = third_line_1.geometry.buffer(13,cap_style='flat')
        buff_4.plot(ax=ax,color = "orange", alpha = 0.5)
        buff_5 = third_line_2.geometry.buffer(13,cap_style='flat')
        buff_5.plot(ax=ax,color = "orange", alpha = 0.5)
        buff_3 = second_line_2.geometry.buffer(13,cap_style='flat')
        buff_3.plot(ax=ax,color = "orange", alpha = 0.5)
    
        third_line_1.plot(ax=ax)
        third_line_2.plot(ax=ax)
        second_line_2.plot(ax=ax)
        counter = 0
        for i in range(len(line_1_vertices)):
            pt = line_1_vertices[(line_1_vertices.index == i)].geometry
            pt = pt.reset_index(drop=True)
            counter += 1
            if (pt.intersects(buff_4)[0] or pt.intersects(buff_5)[0] or pt.intersects(buff_3)[0]):
                pt.plot(ax=ax,color = "grey")
                pt_red_2.append(pt)
            else: 
                pt.plot(ax=ax,color = "red")
                pt_green_2.append(pt)
        pt_green = [pt[0] for pt in pt_green_2]
        pt_red = [pt[0] for pt in pt_red_2]
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        gdf_red = gp.GeoDataFrame(geometry = pt_red)
        gdf_green = gp.GeoDataFrame(geometry = pt_green)
    if count.value == 7:
        print("Compute new lines since there are vertices above length threshold.")
        ax = second_line_2.plot(linestyle = 'dashed',figsize=(12,12))
        third_line_1.plot(ax=ax,linestyle = 'dashed')
        third_line_2.plot(ax=ax,linestyle = 'dashed')
        line_1.plot(color="grey",ax=ax, alpha = 0.5)
        line_1_vertices.plot(ax=ax, color='grey', alpha=0.5)
        
        gdf_green['distance'] = gdf_green.geometry.apply(lambda point: point.distance(third_line_1.geometry[0]))
        farthest_pt_3 = pt_green[1]
        fourth_line_1 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([line_1_vertices.geometry[0],farthest_pt_3])))
        fourth_line_1.plot(ax=ax)
        fourth_line_2 = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([farthest_pt_3,farthest_pt_2])))
        fourth_line_2.plot(ax=ax)
        
        seg_length = np.sqrt(abs(gdf_green['distance'].iloc[1]**2-fourth_line_2.geometry.length.iloc[0]**2))
        perpen_line = gp.GeoDataFrame(geometry=gp.GeoSeries(LineString([farthest_pt_3,line_interpolate_point(third_line_1, -seg_length).geometry[0]])))
        perpen_line.plot(ax=ax,linestyle='dashed')
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    if count.value == 8:
        print("Get the final simplified line thanks to the selected vertices")
        ax = line_1.plot(color="grey",alpha=0.5,figsize=(12,12))
        fourth_line_1.plot(ax=ax)
        fourth_line_2.plot(ax=ax)
        third_line_2.plot(ax=ax)
        second_line_2.plot(ax=ax)
        for i in range(len(line_1_vertices)):
            pt = line_1_vertices[(line_1_vertices.index == i)].geometry
            pt = pt.reset_index(drop=True)
            if pt.touches(fourth_line_1.geometry)[0] or pt.touches(fourth_line_2.geometry)[0] or pt.touches(third_line_2.geometry)[0] or pt.touches(second_line_2.geometry)[0]:
                pt.plot(ax=ax)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)

def next(next_step):
    count.value+=1
        
next_step.on_click(next)

def back(back_step):
    count.value-=1
        
back_step.on_click(back)

out_dp = ipywidgets.interactive(dp,step=count)
display(next_step, back_step, out_dp)

**1.3/** Hands-on

In [ ]:
generate_line = ipywidgets.Button(description='Generate line',button_style='')
show_brit = ipywidgets.Button(description='Show Brittany',button_style='')
length = ipywidgets.IntSlider(value =13, min = 1, max = 5000,step = 1,continuous_update=False)
brit = gp.read_file("data/puntos/brittany.geojson")

def test_dp(length_thshld):
    ax= line_1.plot(color="grey",alpha = 0.7, figsize=(12,12))
    gp.GeoDataFrame(geometry = cg.douglas_peucker(list(line_1.geometry), length_thshld,False)).plot(ax=ax)
    ax.set_aspect('equal')
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

def new_line(generate_line):
    global line_1
    points = []
    for i in range(20):
        points.append((i*1000, random.uniform(*(0, 5000)))) 
    line_1 = LineString(points)
    line_1 = gp.GeoDataFrame(geometry=gp.GeoSeries(line_1))
    if length.value == 13:
        length.value = 12
    else:
        length.value = 13

def show_brit_func(show_brit):
    global line_1
    line_1 = brit
    if length.value == 13:
        length.value = 12
    else:
        length.value = 13
        
generate_line.on_click(new_line)
show_brit.on_click(show_brit_func)

out_test_dp = ipywidgets.interactive(test_dp,length_thshld=length)
display(out_test_dp,generate_line, show_brit)

Although the Douglas-Peucker algorithm is easy to use and understand (only one parameter is required), the resulting simplified geometry often features many sharp angles. This sharp aspect is not ideal when it comes to generalizing natural features, such as rivers or coastlines (as we see with Brittany in our example).

**1.4/** CartAGen's Douglas-Peucker algorithm code details 

In [ ]:
res = cg.douglas_peucker(list(line_1.geometry), threshold=10000 ,preserve_topology=False) #in this example, "line_1" is a GeoDataFrame

gp.GeoDataFrame(geometry = res).plot()

##  **Part 2** : Visvalingam-Whyatt algorithm

This algorithm created by [M. Visvalingam and J.D. Whyatt](https://www.tandfonline.com/doi/abs/10.1179/000870493786962263) in 1993 is an area-based line simplification.

The principle of the algorithm is to select the vertices to delete. To select the vertices to delete, there is an iterative process, and at each iteration, the triangles formed by three consecutive vertices are computed. If the area of the smallest triangle is smaller than a threshold (“area_tolerance” parameter), the middle vertex is deleted, and another iteration starts.

**2.1/** Principle

In [ ]:
line_2 = gp.read_file("data/data_lines/line_2.geojson")
line_2_vertices = gp.GeoDataFrame(geometry=gp.GeoSeries([Point(vertice) for vertice in line_2.geometry[0].coords]))
line_2_vertices = line_2_vertices.set_crs(crs='EPSG:2154')

next_step = ipywidgets.Button(description='Next step',button_style='')
back_step = ipywidgets.Button(description='Previous step',button_style='')
count = ipywidgets.IntText(value=0,min=0,max=10,disabled=True)
count.value = 0

def vw(step):
    global tri
    global pts_tri
    global line_2_vertices
    global line_2_vertices_2
    global line_2_vertices_3
    global line_2_vertices_4

    if count.value == 0:
        ax = line_2.plot(color="grey",figsize=(12,12))
        line_2_vertices.plot(ax=ax,color="grey")
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################    
    if count.value == 1:
        print("Compute the triangle formed by the first three points and calculate its area")
        ax = line_2.plot(color = "grey", figsize = (12,12))
        
        pts_tri = []
        for i in range(3):
            pt = line_2_vertices[(line_2_vertices.index == i)]
            pt.plot(ax=ax)
            pts_tri.append(pt.geometry)
        tri = []
        tri.append(gp.GeoDataFrame(geometry=gp.GeoSeries(Polygon([pts.reset_index(drop=True)[0] for pts in pts_tri]))))
        tri[0].plot(ax=ax,alpha=0.5)
        tri[0]['area'] = tri[0].geometry.area 

        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################
    if count.value == 2:
        print("Repeat this operation for the triangle formed by the 2nd, 3rd and 4th points\nAnd for all the other triangles that follow")
        ax = line_2.plot(color = "black", figsize = (12,12),linewidth=1.5)
        tri= []
        for i in range(len(line_2_vertices)-2):
            pts_tri = []
            for j in range(i,i+3,1):
                pt = line_2_vertices[(line_2_vertices.index == j)]
                pt.plot(ax=ax)
                pts_tri.append(pt.geometry)
                
            tri.append(gp.GeoDataFrame(geometry=gp.GeoSeries(Polygon([Point(point.get_coordinates()) for point in pts_tri]))))
            tri[i].plot(ax=ax,color = (np.random.uniform(0, 1),np.random.uniform(0, 1),np.random.uniform(0, 1)),alpha=0.6)
            tri[i]['area'] = tri[i].geometry.area 

        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################
    if count.value == 3:
        print("Remove the middle vertice of the smallest triangle.\nIf no triangle has an area smaller than the area threshold, the line cannot be simplified.")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha= 0.5)
        smallest_tri = tri[0]
        for i in tri:
            if i.area[0] < smallest_tri.area[0]:
                smallest_tri = i
        if smallest_tri.geometry.area.iloc[0] < 30:
            smallest_tri.plot(ax=ax, color = "pink")
            pt_to_remove = Point(smallest_tri.geometry[0].exterior.coords[1])
            gp.GeoSeries(pt_to_remove).plot(ax=ax, color="red")
        else:
            for i in range(len(line_2_vertices)-2):
                tri[i].plot(ax=ax,color = "lightgreen",alpha=0.8)
        c = -1
        for i in list(line_2_vertices.geometry):
            c += 1
            if i == pt_to_remove:
                line_2_vertices_2 = line_2_vertices.drop(c)
                line_2_vertices_2 = line_2_vertices_2.reset_index(drop = True)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################
    if count.value == 4:
        print("Again, we compute triangles with the new set of vertices.\nAnd measure the area of each triangle")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha=0.5)
        current_line = gp.GeoSeries(LineString([point[1].geometry for point in line_2_vertices_2.iterrows()]))
        current_line.plot(ax=ax, color="black",linewidth=1.5)
        tri= []
        for i in range(len(line_2_vertices_2)-2):
            pts_tri = []
            for j in range(i,i+3,1):
                pt = line_2_vertices_2[(line_2_vertices_2.index == j)]
                pt.plot(ax=ax)
                pts_tri.append(pt.geometry)
                
            tri.append(gp.GeoDataFrame(geometry=gp.GeoSeries(Polygon([Point(point.get_coordinates()) for point in pts_tri]))))
            tri[i].plot(ax=ax,color = (np.random.uniform(0, 1),np.random.uniform(0, 1),np.random.uniform(0, 1)),alpha=0.6)
            tri[i]['area'] = tri[i].geometry.area 

        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################
    if count.value == 5:
        print("Remove the middle vertice of the smallest triangle if it has a surface below the area threshold.")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha= 0.3)
        current_line = gp.GeoSeries(LineString([point[1].geometry for point in line_2_vertices_2.iterrows()]))
        current_line.plot(ax=ax,color="grey")
        smallest_tri = tri[0]
        for i in tri:
            if i.area[0] < smallest_tri.area[0]:
                smallest_tri = i
        if smallest_tri.geometry.area.iloc[0] < 30:
            smallest_tri.plot(ax=ax, color = "pink")
            pt_to_remove = Point(smallest_tri.geometry[0].exterior.coords[1])
            gp.GeoSeries(pt_to_remove).plot(ax=ax, color="red")
        else:
            for i in range(len(line_2_vertices_2)-2):
                tri[i].plot(ax=ax,color = "lightgreen",alpha=0.8)
        c = -1
        for i in list(line_2_vertices_2.geometry):
            c += 1
            if i == pt_to_remove:
                line_2_vertices_3 = line_2_vertices_2.drop(c)
                line_2_vertices_3 = line_2_vertices_3.reset_index(drop = True)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################    
    if count.value == 6:
        print("Repeat the process until there are no vertices to remove")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha=0.5)
        current_line = gp.GeoSeries(LineString([point[1].geometry for point in line_2_vertices_3.iterrows()]))
        current_line.plot(ax=ax, color="black",linewidth=1.5)
        tri= []
        for i in range(len(line_2_vertices_3)-2):
            pts_tri = []
            for j in range(i,i+3,1):
                pt = line_2_vertices_3[(line_2_vertices_3.index == j)]
                pt.plot(ax=ax)
                pts_tri.append(pt.geometry)
                
            tri.append(gp.GeoDataFrame(geometry=gp.GeoSeries(Polygon([Point(point.get_coordinates()) for point in pts_tri]))))
            tri[i].plot(ax=ax,color = (np.random.uniform(0, 1),np.random.uniform(0, 1),np.random.uniform(0, 1)),alpha=0.6)
            tri[i]['area'] = tri[i].geometry.area 
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################
    if count.value == 7:
        print("Repeat the process until there are no vertices to remove")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha= 0.3)
        current_line = gp.GeoSeries(LineString([point[1].geometry for point in line_2_vertices_3.iterrows()]))
        current_line.plot(ax=ax,color="grey")
        smallest_tri = tri[0]
        for i in tri:
            if i.area[0] < smallest_tri.area[0]:
                smallest_tri = i
        if smallest_tri.geometry.area.iloc[0] < 30:
            smallest_tri.plot(ax=ax, color = "pink")
            pt_to_remove = Point(smallest_tri.geometry[0].exterior.coords[1])
            gp.GeoSeries(pt_to_remove).plot(ax=ax, color="red")
        else:
            for i in range(len(line_2_vertices_3)-2):
                tri[i].plot(ax=ax,color = "lightgreen",alpha=0.8)
        c = -1
        for i in list(line_2_vertices_3.geometry):
            c += 1
            if i == pt_to_remove:
                line_2_vertices_4 = line_2_vertices_3.drop(c)
                line_2_vertices_4 = line_2_vertices_4.reset_index(drop = True)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################    
    if count.value == 8:
        print("Repeat the process until there are no vertices to remove")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha=0.5)
        current_line = gp.GeoSeries(LineString([point[1].geometry for point in line_2_vertices_4.iterrows()]))
        current_line.plot(ax=ax, color="black",linewidth=1.5)
        tri= []
        for i in range(len(line_2_vertices_4)-2):
            pts_tri = []
            for j in range(i,i+3,1):
                pt = line_2_vertices_4[(line_2_vertices_4.index == j)]
                pt.plot(ax=ax)
                pts_tri.append(pt.geometry)
                
            tri.append(gp.GeoDataFrame(geometry=gp.GeoSeries(Polygon([Point(point.get_coordinates()) for point in pts_tri]))))
            tri[i].plot(ax=ax,color = (np.random.uniform(0, 1),np.random.uniform(0, 1),np.random.uniform(0, 1)),alpha=0.6)
            tri[i]['area'] = tri[i].geometry.area 
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################
    if count.value == 9:
        print("In this case, there are no triangles below the area threshold.\nThus the line can't be more simplified given this threshold")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha= 0.3)
        current_line = gp.GeoSeries(LineString([point[1].geometry for point in line_2_vertices_4.iterrows()]))
        current_line.plot(ax=ax,color="grey")
        smallest_tri = tri[0]
        for i in tri:
            if i.area[0] < smallest_tri.area[0]:
                smallest_tri = i
        if smallest_tri.geometry.area.iloc[0] < 30:
            smallest_tri.plot(ax=ax, color = "pink")
            pt_to_remove = Point(smallest_tri.geometry[0].exterior.coords[1])
            gp.GeoSeries(pt_to_remove).plot(ax=ax, color="red")
        else:
            for i in range(len(line_2_vertices_4)-2):
                tri[i].plot(ax=ax,color = "lightgreen",alpha=0.8)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ##############################################    
    if count.value == 10:
        print("Compute the final line")
        ax = line_2.plot(color = "grey", figsize = (12,12), alpha=0.5)
        current_line = gp.GeoSeries(LineString([point[1].geometry for point in line_2_vertices_4.iterrows()]))
        current_line.plot(ax=ax, color="black",linewidth=1.5)
        line_2_vertices_4.plot(ax=ax,color="black")
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)

        

def next(next_step):
    count.value+=1
        
next_step.on_click(next)

def back(back_step):
    count.value-=1
        
back_step.on_click(back)

out_vw = ipywidgets.interactive(vw,step=count)
display(next_step, back_step, out_vw)

**2.2/** Hands-on

In [ ]:
generate_line = ipywidgets.Button(description='Generate line',button_style='')
show_river = ipywidgets.Button(description='Show river',button_style='')
area = ipywidgets.IntSlider(value =13, min = 1, max = 5000,step = 1,continuous_update=False)
river = gp.read_file("data/data_lines/river.geojson")

def test_vw(area_threshold):
    ax= line_2.plot(color="grey",alpha = 0.7, figsize=(10,10))
    gp.GeoDataFrame(geometry = gp.GeoSeries(cg.visvalingam_whyatt(line_2.geometry[0], area_threshold))).plot(ax=ax)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

def new_line(generate_line):
    global line_2
    points = []
    for i in range(30):
        points.append((i*10, random.uniform(*(0, 100)))) 
    line_2 = LineString(points)
    line_2 = gp.GeoDataFrame(geometry=gp.GeoSeries(line_2))
    if area.value == 13:
        area.value = 12
    else:
        area.value = 13

def show_river_func(show_river):
    global line_2
    line_2 = river[(river.index == 19)]
    line_2 = line_2.reset_index(drop=True)
    if area.value == 13:
        area.value = 12
    else:
        area.value = 13
        
generate_line.on_click(new_line)
show_river.on_click(show_river_func)

out_test_vw = ipywidgets.interactive(test_vw,area_threshold=area)
display(out_test_vw,generate_line, show_river)

**2.3/** CartAGen's Visvalingam-Whyatt algorithm code details

In [ ]:
res = cg.visvalingam_whyatt(line_2.geometry[0], area_tolerance=30) #in this example, "line_1" is a GeoDataFrame

ax= gp.GeoDataFrame(geometry = gp.GeoSeries(res)).plot(figsize=(10,10))
line_2.plot(ax=ax, color="grey", alpha=0.5)

The main drawback of this algorithm is its computation time, which is longer than that of the Douglas-Peucker algorithm.

However, it tends to give less angular results, making it more relevant for simplifying natural lines or polygons such as rivers, forests or coastlines.

## **Part 3** Li-Openshaw algorithm

This algorithm created by [Zhilin Li and Stan Openshaw](https://api.semanticscholar.org/CorpusID:120732702) in 1993 is based on a regular square grid.

It first divide the line vertices into groups partionned by a regular grid, then each group of vertexes is replaced by their centroid.

**3.1/** Principle

In [ ]:
next_step = ipywidgets.Button(description='Next step',button_style='')
back_step = ipywidgets.Button(description='Previous step',button_style='')
count = ipywidgets.IntText(value=0, min=0,max=4,disabled=True)
line_1 = gp.read_file("data/data_lines/ligne.geojson")
line_1_vertices = gp.GeoDataFrame(geometry=gp.GeoSeries([Point(vertice) for vertice in line_1.geometry[0].coords]))

def lo(step):
    global grid
    global pts_to_keep
    global idx
    if count.value == 0:
        ax = line_1.plot(color="grey",figsize=(12,12))
        line_1_vertices.plot(ax=ax,color="grey")
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################
    if count.value == 1:
        print("Compute a regular square grid based on the extent of the line.\nThe size of the grid cells is parametrable")
        ax = line_1.plot(color="grey",figsize=(12,12))
        line_1_vertices.plot(ax=ax,color="grey",alpha=0.5)
        idx, grid = cg.partition_grid(line_1_vertices,20)
        gp.GeoDataFrame(geometry=grid).plot(ax=ax,facecolor = 'None',edgecolor="darkblue")
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################    
    if count.value == 2:
        print("Inside each cell, retrieve the line vertices and compute their centroid.\nThose centroids will be the vertices of the simplified line")
        ax = line_1_vertices.plot(color="grey", figsize=(12,12))
        gp.GeoDataFrame(geometry=grid).plot(ax=ax,facecolor = 'None',edgecolor="darkblue")
        pts_to_keep = []
        for i in idx:
            if len(i) > 0:
                pts_to_compute = []
                for j in i:
                    pts_to_compute.append(line_1_vertices.iloc[j].iloc[0])
                pts_to_compute = MultiPoint(pts_to_compute)
                pt_keep = gp.GeoSeries(centroid(pts_to_compute))
                pt_keep.plot(ax=ax, color = "green")
                pts_to_keep.append(pt_keep)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################    
    if count.value == 3:
        print("Make sure the first and last points of the original line are kept aswell")
        ax = line_1_vertices.plot(color="grey", figsize=(12,12),alpha = 0.3)
        gp.GeoDataFrame(geometry=grid).plot(ax=ax,facecolor = 'None',edgecolor="darkblue",alpha=0.5)
        first = gp.GeoSeries(line_1_vertices.iloc[0].geometry)
        first.plot(ax=ax, color = "green")
        last = gp.GeoSeries(line_1_vertices.iloc[len(line_1_vertices)-1].geometry)
        last.plot(ax=ax, color = "green")
        gp.GeoDataFrame(geometry=gp.GeoSeries([point[0] for point in pts_to_keep])).plot(ax=ax,color="grey")
        pts_to_keep.insert(0,first)
        pts_to_keep.insert(len(pts_to_keep),last)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################    
    if count.value == 4:
        print("Compute the simplified line")
        ax = line_1.plot(color="grey",figsize=(12,12),alpha=0.5)
        gp.GeoDataFrame(geometry=gp.GeoSeries([point[0] for point in pts_to_keep])).plot(ax=ax)
        pts_to_keep[7], pts_to_keep[8] = pts_to_keep[8], pts_to_keep[7]
        final_line = gp.GeoSeries(LineString([point[0] for point in pts_to_keep]))
        final_line.plot(ax=ax)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)

def next(next_step):
    count.value+=1
        
next_step.on_click(next)

def back(back_step):
    count.value-=1
        
back_step.on_click(back)

out_lo = ipywidgets.interactive(lo,step=count)
display(next_step, back_step, out_lo)

**3.2/** Hands-on

In [ ]:
generate_line = ipywidgets.Button(description='Generate line',button_style='')
show_river = ipywidgets.Button(description='Show river',button_style='')
cell = ipywidgets.IntSlider(value =13, min = 1, max = 600,step = 1,continuous_update=False)
river = gp.read_file("data/data_lines/river.geojson")

def test_lo(cell_size):
    ax= line_1.plot(color="grey",alpha = 0.5, figsize=(10,10))
    gp.GeoSeries(cg.li_openshaw(force_2d(line_1.geometry[0]),cell_size)).plot(ax=ax)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

def new_line(generate_line):
    global line_1
    points = []
    for i in range(30):
        points.append((i*10, random.uniform(*(0, 100)))) 
    line_1 = LineString(points)
    line_1 = gp.GeoDataFrame(geometry=gp.GeoSeries(line_1))
    if cell.value == 13:
        cell.value = 12
    else:
        cell.value = 13

def show_river_func(show_river):
    global line_1
    line_1 = river[(river.index == 19)]
    line_1 = line_1.reset_index(drop=True)
    if cell.value == 13:
        cell.value = 12
    else:
        cell.value = 13
        
generate_line.on_click(new_line)
show_river.on_click(show_river_func)

out_test_lo = ipywidgets.interactive(test_lo,cell_size=cell)
display(out_test_lo,generate_line, show_river)


**3.3/** CartAGen's Li-Openshaw algorithm code details

In [ ]:
ax= line_1.plot(color="grey",alpha = 0.5, figsize=(10,10))
    
gp.GeoSeries(cg.li_openshaw(force_2d(line_1.geometry[0]),35)).plot(ax=ax)

Like the Visvalingam-Whyatt algorithm, the Li-Openshaw algorithm seems better suited to simplifying natural features such as rivers. It also has the advantage of being faster in terms of computation time.

However, the fact that the vertices of the simplified line are not necessarily part of the original line can pose a problem and lead to unrealistic results in certain situations.

## **Part 4** Raposo algorithm

This algorithm created by [Paulo Raposo](https://www.tandfonline.com/doi/abs/10.1080/15230406.2013.803707) in 2013 is similar to the Li-Openshaw algorithm, since it involves a regular grid.

The idea of the algorithm is to put a hexagonal tessallation on top of the line to simplify, the size of the cells depending on the targeted granularity of the line. Similarly to the Li-Openshaw algorithm, only one vertex is kept inside each cell. This point can be the centroid of the removed vertices, or a projection on the initial line of this centroid.

**4.1/** Principle

In [ ]:
next_step = ipywidgets.Button(description='Next step',button_style='')
back_step = ipywidgets.Button(description='Previous step',button_style='')
count_rapo = ipywidgets.IntText(value=0,min=0,max=4,disabled=True)
line_1 = gp.read_file("data/data_lines/ligne.geojson")
line_1_vertices = gp.GeoDataFrame(geometry=gp.GeoSeries([Point(vertice) for vertice in line_1.geometry[0].coords]))

def rapo(step):
    global hexgrid
    global pts_to_keep
    global idx
    if count_rapo.value == 0:
        ax = line_1.plot(color="grey",figsize=(12,12))
        line_1_vertices.plot(ax=ax,color="grey")
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################
    if count_rapo.value == 1:
        print("Compute a hexagonal grid based on the extent of the line.\nThe size of side of the grid cells is the result of this function :\naverage length of line segments * generalization factor, i.e (1/final scale)/(1/initial scale)")
        ax = line_1.plot(color="grey",figsize=(12,12))
        line_1_vertices.plot(ax=ax,color="grey",alpha=0.5)
        hexgrid = cg.tessellate(gp.GeoSeries(line_1.geometry[0].envelope).total_bounds, 19.7, shape='hexagonal')
        gp.GeoDataFrame(geometry=hexgrid).plot(ax=ax,facecolor = 'None',edgecolor="darkblue")
        plt.xlim(gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[0]-20, gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[2]+20)
        plt.ylim(gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[1]-20, gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[3]+20)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################    
    if count_rapo.value == 2:
        print("Inside each cells, retrieve the line vertices and compute their centroid.\nThose centroids will be the vertices of the simplified line")
        ax = line_1_vertices.plot(color="grey", figsize=(12,12))
        gp.GeoDataFrame(geometry=hexgrid).plot(ax=ax,facecolor = 'None',edgecolor="darkblue",alpha=0.5)
        pts_to_keep = []
        for i in range(len(hexgrid)):
            gdf = gp.GeoDataFrame(geometry=intersection(gp.GeoSeries(hexgrid)[i], line_1_vertices.geometry))
            gdf = gdf[~gdf.is_empty]
            if len(gdf) > 0:
                pts_to_compute = []
                for j,row in gdf.iterrows():
                    pts_to_compute.append(line_1_vertices.iloc[j].iloc[0])
                pts_to_compute = MultiPoint(pts_to_compute)
                pt_keep = gp.GeoSeries(centroid(pts_to_compute))
                pt_keep.plot(ax=ax, color = "green")
                pts_to_keep.append(pt_keep)
        plt.xlim(gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[0]-20, gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[2]+20)
        plt.ylim(gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[1]-20, gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[3]+20)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################    
    if count_rapo.value == 3:
        print("Make sure the first and last points of the original line are kept aswell")
        ax = line_1_vertices.plot(color="grey", figsize=(12,12),alpha = 0.3)
        gp.GeoDataFrame(geometry=hexgrid).plot(ax=ax,facecolor = 'None',edgecolor="darkblue",alpha=0.5)
        # first = gp.GeoSeries(line_1_vertices.iloc[0].geometry)
        # first.plot(ax=ax, color = "green")
        gp.GeoDataFrame(geometry=gp.GeoSeries([point[0] for point in pts_to_keep])).plot(ax=ax,color="grey")
        last = gp.GeoSeries(line_1_vertices.iloc[len(line_1_vertices)-1].geometry)
        last.plot(ax=ax, color = "green")
        #pts_to_keep.insert(0,first)
        pts_to_keep.insert(len(pts_to_keep),last)
        plt.xlim(gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[0]-20, gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[2]+20)
        plt.ylim(gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[1]-20, gp.GeoSeries(line_1.geometry[0].envelope).total_bounds[3]+20)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
    ###############################################    
    if count_rapo.value == 4:
        print("Compute the simplified line")
        ax = line_1.plot(color="grey",figsize=(12,12),alpha=0.5)
        gp.GeoDataFrame(geometry=gp.GeoSeries([point[0] for point in pts_to_keep])).plot(ax=ax)
        #pts_to_keep[7], pts_to_keep[8] = pts_to_keep[8], pts_to_keep[7]
        final_line = gp.GeoSeries(LineString([point[0] for point in pts_to_keep]))
        final_line.plot(ax=ax)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)

def next(next_step):
    count_rapo.value+=1
        
next_step.on_click(next)

def back(back_step):
    count_rapo.value-=1
        
back_step.on_click(back)

out_rapo = ipywidgets.interactive(rapo,step=count_rapo)
display(next_step, back_step, out_rapo)

**4.2/** Hands on

In [ ]:
generate_line = ipywidgets.Button(description='Generate line',button_style='')
show_river = ipywidgets.Button(description='Show river',button_style='')
initial_scale = ipywidgets.IntText(value=1000)
final_scale = ipywidgets.IntText(value=2500)
in_line = ipywidgets.Checkbox(value=True, description="Create new points")  
river = gp.read_file("data/data_lines/river.geojson")

def test_rapo(initial_scale, final_scale, in_line):
    ax= line_1.plot(color="grey",alpha = 0.5, figsize=(10,10))
    gp.GeoSeries(cg.raposo(line_1.geometry[0], initial_scale, final_scale, centroid=in_line, tobler=False)).plot(ax=ax)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

def new_line(generate_line):
    points = []
    for i in range(30):
        points.append((i*10, random.uniform(*(0, 100)))) 
    line_1 = LineString(points)
    line_1 = gp.GeoDataFrame(geometry=gp.GeoSeries(line_1))
    if initial_scale.value == 13:
        initial_scale.value = 12
    else:
        initial_scale.value = 13

def show_river_func(show_river):
    line_1 = river[(river.index == 19)]
    line_1 = line_1.reset_index(drop=True)
    if initial_scale.value == 13:
        initial_scale.value = 12
    else:
        initial_scale.value = 13
        
generate_line.on_click(new_line)
show_river.on_click(show_river_func)

out_test_rapo = ipywidgets.interactive(test_rapo,initial_scale=initial_scale, final_scale=final_scale, in_line=in_line)
display(out_test_rapo,generate_line, show_river)


**4.3/** CartAGen's Raposo algorithm code details

In [ ]:
ax= line_1.plot(color="grey",alpha = 0.5, figsize=(10,10))
    
res = gp.GeoSeries(cg.raposo(line_1.geometry[0], initial_scale=1000, final_scale=2000, centroid=True, tobler=False))
res.plot(ax=ax)

Although the computation time is longer for this algorithm than for the others, it offers several advantages :

- Firstly, like Li-Openshaw and Visvalingam-Whyatt, it tends to give less angular results, making it suitable for simplifying natural features.
- Secondly, unlike Li-Openshaw, it is possible to configure the algorithm so that the vertices of the simplified line touch the initial line. It avoids the lack of realism that can occur when creating vertices independent of this line.
- Finally, the simplification strength is derived from the initial and output scale parameters. This is particularly useful when the scale of the initial data and of the target map is known.

Thank you for your attention !